In [1]:
#process data

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from problem_config import *
df = pd.read_parquet("../data/raw_data/phase-1/prob-1/raw_train.parquet")

# Drop vì thấy feature1 này nhảm nhí
df.drop(["feature1"], inplace=True, axis=1)

# encode feature2
le = LabelEncoder()
le.fit(pd.unique(df["feature2"]))
f2_encode = le.transform(df["feature2"])
df["feature2"] = f2_encode
df[:3]
from sklearn.model_selection import train_test_split
y_data = df["label"]
x_data = df.drop("label", axis=1)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.3)
# from sklearn.linear_model import LogisticRegression
# model = LogisticRegression()



In [3]:
# register model to mlflow
from mlflow.models.signature import infer_signature
import mlflow
# init mlflow
mlflow.set_tracking_uri('http://localhost:5000')
mlflow.set_experiment(
    f"phase-1_prob-1_model-1"
)

2023/06/18 17:36:25 INFO mlflow.tracking.fluent: Experiment with name 'phase-1_prob-1_model-1' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1687109785650, experiment_id='3', last_update_time=1687109785650, lifecycle_stage='active', name='phase-1_prob-1_model-1', tags={}>

In [4]:
# train model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train, y_train)


RandomForestClassifier()

In [5]:
# evaluate model
from sklearn.metrics import classification_report, roc_auc_score
predictions = model.predict(x_test)
print(classification_report(y_test, predictions))
auc_score = roc_auc_score(y_test, predictions)
metrics = {"test_auc": auc_score}
print(f"metrics: {metrics}")

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     37695
           1       0.98      0.91      0.94      2566

    accuracy                           0.99     40261
   macro avg       0.98      0.95      0.97     40261
weighted avg       0.99      0.99      0.99     40261

metrics: {'test_auc': 0.9547902789102797}


In [6]:
# mlflow log and end experiment
mlflow.log_params(model.get_params())
mlflow.log_metrics(metrics)
signature = infer_signature(x_test, predictions)
mlflow.sklearn.log_model(
    sk_model=model,
    artifact_path='model',
    signature=signature,
)
mlflow.end_run()

/home/iamqh/miniconda3/lib/python3.10/site-packages/mlflow/models/signature.py:137: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input)
/home/iamqh/miniconda3/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
